In [1]:
import pandas as pd
import numpy as np  
import tensorflow as tf

In [25]:
#Load trained model, scaler and encoders

import pickle
from tensorflow.keras.models import load_model

model =load_model("churn_prediction_model.h5")  # Load the trained model

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)  # Load the scaler
    
with open("label_encoder.pkl",'rb') as file:
    label_encoder = pickle.load(file)       # Load the label encoder
    
with open("onehot_encoder.pkl",'rb') as file:
    onehot = pickle.load(file)  # Load the one-hot encoder

Making Predictions

In [4]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [26]:
geo_encoded = onehot.transform([[input_data['Geography']]])  # One-hot encode the Geography feature
geo_encoded_df = pd.DataFrame(geo_encoded.toarray(), columns=onehot.get_feature_names_out(['Geography']))

e:\Github Repositories\Churn-Prediction-Model\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [27]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [28]:
input_df = pd.concat([input_df.drop('Geography', axis=1), geo_encoded_df], axis=1)  # Combine the input data with the encoded Geography feature 
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [29]:
input_df['Gender'] = label_encoder.transform([input_df['Gender']])
input_df

e:\Github Repositories\Churn-Prediction-Model\venv\Lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [30]:
scaled_input = scaler.transform(input_df)

In [31]:
prediction = model.predict(input_df)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step


array([[1.]], dtype=float32)

In [32]:
pred_prob = prediction[0][0]

In [33]:
if pred_prob > 0.5:
    print("The customer is likely to churn.")
else:
    print("The customer is likely to stay in the Bank.")

The customer is likely to churn.
